In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from matplotlib import pyplot
from numpy import concatenate
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.python.keras.layers import Dense, LSTM , Dropout, CuDNNLSTM
from tensorflow.python.keras import Sequential
from math import sqrt

In [ ]:
TIME_STEPS = 4
NUM_FEATURES = 4
TOTAL_FEATURES = TIME_STEPS * NUM_FEATURES

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**DATASET PREPROCESSING**




Dataset Loading and dropping the rows with NA values.Plotting the graph for each columns. Further preprocessing includes:




1. Feature Selection
2. Label Encoding
3. Check correlation
4. Converting to Time Series data

In [ ]:
#dataset preprocessing
#datset loading dropping na values

dataset= pd.read_csv('../input/beijing-pm25-data-data-set/PRSA_data_2010.1.1-2014.12.31.csv');
dataset=dataset.dropna();

# **1. Feature Selection**
There are 8 features important for the forecast: PM2.5, dew point, temperature, pressure, wind direction, wind speed and the cumulative number of hours of snow and rain. Hence, dropping other features.

In [ ]:
dataset=dataset.drop('No',axis=1);
dataset=dataset.drop('year',axis=1);
dataset=dataset.drop('month',axis=1);
dataset=dataset.drop('day',axis=1);
dataset=dataset.drop('hour',axis=1);
dataset.head();

values=dataset.values;

In [ ]:
dataset.head()

In [ ]:
# specify columns to plot
groups = [0, 1, 2, 3, 5, 6, 7]
i = 1
# plotting each column
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(dataset.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()

# **2.** **Label** **Encoding**
The Wind direction doesn’t contain numerical values so label encoding is done.


In [ ]:
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')

In [ ]:
values.shape

# **3. Check Correlation**

In [ ]:
corr = np.corrcoef(values.T)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = pyplot.subplots(figsize=(7, 5))
    ax = sns.heatmap(corr, mask=mask,annot=True, vmax=.3, square=True)

In [ ]:
values = np.delete(values, [1,3,6,7], 1)

# **4. Converting to Time Series Data**
Since we use the LSTM neural network, we must sort the data according to the time. The dataset is transformed into a supervised learning problem. The weather variables for the hour to be predicted (t) are then removed. So, we have features for previous timestep (t-1) and for prediction of pollution PM2.5 taking its current timestep (t) data.

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(values, TIME_STEPS, 1)
#drop columns we don't want to predict
rem_cols = list(range(reframed.shape[1]-NUM_FEATURES+1,reframed.shape[1]))
reframed.drop(reframed.columns[rem_cols], axis=1, inplace=True)
print(reframed.head())

In [ ]:
assert reframed.shape[1] == NUM_FEATURES * TIME_STEPS + 1

# Data Normalization
Data Normalization is done using MinMaxScaler function of sklearn.

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(reframed)

# Splitting into train and test sets
We will only fit the model on the first 2 years(365* 24 * 2 hours) of data, then evaluate it on the remaining 3 years of data. 

In [ ]:
# split into train and test sets
values = scaled
n_train_hours = 365 * 24*4
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :TOTAL_FEATURES], train[:, TOTAL_FEATURES]
test_X, test_y = test[:, :TOTAL_FEATURES], test[:, TOTAL_FEATURES]

# reshaping input to be 3D [samples, timesteps, features]

(17520, 1, 8) (17520,) (24236, 1, 8) (24236,)


In [ ]:
train_X = train_X.reshape((train_X.shape[0], TIME_STEPS, NUM_FEATURES))
test_X = test_X.reshape((test_X.shape[0], TIME_STEPS, NUM_FEATURES))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
train_X[0]

# **Designing Network**
We will define the LSTM with 100 neurons in the first hidden layer and a Dropout Layer of 0.3,Next there will be another hidden layer of 50 neurons and a Dropout of 0.2. Similarly, there will two more hidden layers with respective 0.2 Dropouts and 50 neurons. In Final Layer, 1 neuron in the output layer for predicting pollution. In the activation Function we used linear function, because of sequential dataset. In the batch size we used three days (24*3 Hours) data. Optimizer function we used Adam method. Loss function we used mean squared error. By monitoring the value of test data loss function, stop the training model when it is not decreasing, and save the current best model.

In [ ]:
if tf.test.is_gpu_available():
    lstm_layer = CuDNNLSTM
else:
    import functools
    lstm_layer = functools.partial(
            LSTM, recurrent_activation='sigmoid')

In [ ]:
# design network
model = Sequential()
model.add(LSTM(100, return_sequences = True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.3))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(1,activation='linear'))

model.compile(loss='mse', optimizer='adam')

In [ ]:
model.summary()

# **Fitting the Network**
Network is fit with epochs size of 50 , batch size of 72.

In [ ]:
# fit network
earlystop_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_loss',
  patience=10)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('air_poll_lstm_3_ts.h5', monitor='val_loss', verbose=1, save_best_only=True)
history = model.fit(train_X, train_y, epochs=50, callbacks=[model_checkpoint, earlystop_callback],
                    batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# **Plotting the graph of Train Loss and Test Loss**

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

# **Making Prediction/Forecasting**
Forecasting the results and invert the scaling of the prediction and test data to check.

In [ ]:
# make a prediction
yhat = model.predict(test_X)
m_test_X = test_X.reshape((test_X.shape[0], TOTAL_FEATURES))
print(yhat.shape, m_test_X.shape)

In [ ]:
# invert scaling for forecast
inv_yhat = concatenate((yhat, m_test_X[:, :]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, m_test_X[:, :]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# **Calculating the RMSE and MAE values**

Test RMSE: 27.188
Test MAE: 17.932

In [ ]:
# calculate RMSE and MAE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
mae = (mean_absolute_error(inv_y, inv_yhat))
print('Test MAE: %.3f' % mae)

# **Plotting the Graph of Actual vs Predicted**

In [ ]:
print('Actual :', inv_y)
print('Predicted:', inv_yhat)
# plot history
pyplot.plot(inv_y, label='Actual')
pyplot.plot(inv_yhat, label='Predicted')
pyplot.legend()
pyplot.show()